In [3]:
# 연습문제 :서울시 응답소 사이트의 원순씨에게 바랍니다 게시판 크롤링하기
# https://eungdapso.seoul.go.kr/

#Step 1. 필요한 모듈과 라이브러리를 로딩합니다.

from bs4 import BeautifulSoup
from selenium import webdriver
import time
import sys
import re
import math
import numpy  
import pandas as pd  
import xlwt

import random
import os   # 폴더 생성하기 위해 필요함

#Step 2. 사용자에게 크롤링에 필요한 정보를 입력 받습니다.
print("=" *100)
print("연습문제 : 서울시 응답소 게시판 크롤링하기")
print("=" *100)

query_txt = '서울시응답소'


cnt = int(input('1.크롤링 할 건수는 몇건입니까?: '))
page_cnt = math.ceil(cnt / 10)

f_dir = input("2.결과 파일을 저장할 폴더명만 쓰세요(예:c:\\temp\\):")
if f_dir=='' :
    f_dir ='c:\\temp\\'

# 저장될 파일위치와 이름을 지정합니다
now = time.localtime()
s = '%04d-%02d-%02d-%02d-%02d-%02d' % (now.tm_year, now.tm_mon, now.tm_mday, now.tm_hour, now.tm_min, now.tm_sec)

os.makedirs(f_dir+s+'-'+query_txt)
os.chdir(f_dir+s+'-'+query_txt)

ff_name=f_dir+s+'-'+query_txt+'\\'+s+'-'+query_txt+'.txt'
fc_name=f_dir+s+'-'+query_txt+'\\'+s+'-'+query_txt+'.csv'
fx_name=f_dir+s+'-'+query_txt+'\\'+s+'-'+query_txt+'.xls'

#Step 3. 크롬 드라이버를 사용해서 웹 브라우저를 실행합니다.

s_time = time.time( )

path = "c:/temp/chromedriver_85/chromedriver.exe"
driver = webdriver.Chrome(path)

driver.get('https://eungdapso.seoul.go.kr/')

time.sleep(random.randrange(2,5))  # 2 - 5 초 사이에 랜덤으로 시간 선택

#Step 4. 여러 메뉴 중에서 시장에게 바랍니다 게시판 선택
driver.find_element_by_xpath("""//*[@id="gnb"]/div[4]/div[1]/div/ul/li[6]/a""").click()

#시간을 계산하기 위해 현재 타임스탬프 찍기
now1 = time.localtime()
s1 = '%04d-%02d-%02d-%02d:%02d:%02d' % (now1.tm_year, now1.tm_mon, now1.tm_mday, now1.tm_hour, now1.tm_min, now1.tm_sec)

print("\n")
page_cnt = math.ceil(cnt / 10)
print("크롤링 할 총 페이지 번호: ",page_cnt)
print("=" *80)

# 페이지 번호 만들기
page_no=[]

for i in range(5,cnt) :
    if i % 5 == 0 :
            page_no.append(i + 1)
                
# Step 5.게시물의 정보 수집

no2=[]             # 게시글 번호 컬럼
contents=[]        # 게시글 내용 컬럼

click_cnt=0

no = 1

for x in range(1,page_cnt+1) :
    print("%s 페이지 내용 수집 시작합니다 =======================" %x)
        
    # 1 페이지에 있는 게시글 10건의 상세 내역 가져오기
    for i in range(1,11) :
        
        if no > cnt :
            break
            
        print
        f = open(ff_name, 'a',encoding='UTF-8')
        
        driver.find_element_by_xpath("""//*[@id="content_cont"]/div[2]/div/form/div[4]/div[%s]/ul/li[2]/a"""  %i).click( )
                               
        html = driver.page_source
        soup = BeautifulSoup(html, 'html.parser')
        
        content = soup.find('div','table_inner_desc').get_text().replace("\t","")
        f.write(str(no)+':'+str(content)+'\n')
        f.close( )
        
        print(no,':',content)
   
        no2.append(no)                             # 게시글 번호에 추가
        contents.append(content.replace("\n","")+'\n')   # 내용에 추가
        
        no += 1
        
        time.sleep(1)
        
        driver.back()
        
    time.sleep(2)        # 페이지 변경 전 2초 대기 

    i = math.floor(x/5) -1
    
    if x > page_cnt+1 :
        break           
    
    if no > cnt :
        break
    elif x == page_no[i]:
        driver.find_element_by_link_text('''다음''').click()
    else :
        driver.find_element_by_link_text("""%s""" %x).click() # 다음 페이지번호 클릭

    
# Step 6. 수집된 정보를 다양한 형식의 파일로 저장하기 
seoul = pd.DataFrame()
seoul['번호']=no2
seoul['상담내용']=contents

# csv 형태로 저장하기
seoul.to_csv(fc_name,encoding="utf-8-sig",index=False)

# 엑셀 형태로 저장하기
seoul.to_excel(fx_name,index=False)

#Step 7. 요약 정보 출력하기
e_time = time.time( )     # 검색이 종료된 시점의 timestamp 를 지정합니다
t_time = e_time - s_time

print("\n") 
print("=" *100)
print("총 소요시간은 %s 초 입니다 " %round(t_time,1))
print("파일 저장 완료: txt 파일명 : %s " %ff_name)
print("파일 저장 완료: csv 파일명 : %s " %fc_name)
print("파일 저장 완료: xls 파일명 : %s " %fx_name)
print("=" *100)

driver.close( )

연습문제 : 서울시 응답소 게시판 크롤링하기
1.크롤링 할 건수는 몇건입니까?: 50
2.결과 파일을 저장할 폴더명만 쓰세요(예:c:\temp\):


크롤링 할 총 페이지 번호:  5
1 페이지 내용 수집 시작합니다 =======================
1 : 
Q. 상담내용
                                    해당내용 글을 올렸으나 아직도 황당하고 어이가없어 여기에 글을 올립니다 탈락 된 내용조차 확인하기위해 홈페이지 이곳저곳을 확인했고 분명 심사통과되서 1순위확인까지하고기다렸는데 탈락햇다는 내용도 못보고 이의신청도 하지못햇습니다. 당연히 될거라고생각하고잇는데 어째서 탈락인지 이해가 안되고 이의신청도 할수없게 아무런연락도 안주시고 광고는대문짝만하게해놓고 정작 탈락이 말이되나요여기저기곳곳에서 문제에대한 건의가많은데 시간내서 오류도많았던 사이트에서힘들게올렸는데 ., , 대체 어떤청년에게 월세를 지원하시는겁니까?
                                
2 : 
Q. 상담내용
                                    평화의 공원 잔디광장을 축구장으로 만드다고 나랏돈으로 여론을 조성하고 엉뚱한 짓을 하는 못된 인간이 있다고 들었습니다.  누구에 의해 발의되고 추진되고있는지 그 현황과 그간 소요된 비용등에 대하여 알려주세요 그리고 모든시민이 이용하는 잔디광장을 축구동호회 소수인원만 사용하게하고 또 그렇게 추진하는 과정에 쓸데없이 소요되는 막대하게 소요될 예산도 알려주세요
                                
3 : 
Q. 상담내용
                                    안녕하세요. 저는 이번주 결혼식을 예정하고 있는 신부입니다. 저는 지금 1년을 넘게 설레는 마음으로 준비를 한 결혼식이 코로나라는 국가적 재난상황을 맞이하여 진행이 불가피하게 되는 상황에 처해있습니다. 지금 제가 겪고 있는 것이 저희 부부의 일만이 아닌 많은 신혼부부들과 결혼을 꿈

10 : 
Q. 상담내용
                                    안녕하십니까? 해마다 우기가 되면 호우경보가 울릴까 봐, 묵동천이 넘칠까 조마조마 마음 졸이고 잠을 설치며 살고 있는 중랑구 묵동 주민입니다.  태릉골프장 아래에 있는 묵동천은 집중호우 때 물의 양이 급속도로 증가되어 수위가 순식간에 상승되는 지역입니다. 빗물이 묵동천 제방 위협할 때가 여러 번 있었으며 묵동천 제방 아래 지역은 하수 역류 등으로 집중호우 때  침수 피해가 조금 있었던 적도 있습니다.  골프장의 잔디와 흙은 빗물을 흡수하여 땅속에 저장하는 중요한 역할을 하고 있습니다. 태릉골프장 개발이 되어 일만 채 이상의 아파트 건물이 들어서면 아스팔트 포장으로 빗물이 땅속으로 스며들지 못해 많은 양의 빗물이 순식간에 흘러 들어와 묵동천이 감당하지 못해 넘칠 것입니다.  또한 일만 채의 아파트에서 동시에 쏟아져 나오는 생활하수의 양도 엄청나서 집중 호우 시  제방을 위협할 것이라 생각됩니다.  지금 한창 개발되고 있는 신내역 주변 아파트도 내년부터 입주를 합니다. 이곳 역시 많은 생활하수와 빗물의 양이 증가하여 묵동천을 위협할 것입니다. 설상가상 태릉 골프장까지 같이 개발된다면 지금도 간신히 버티는 묵동천은 물의 양(생활하수+빗물)을 감당하기 어려워 둑이 넘치고 무너질 것입니다. 공릉동 묵동 주민들 피해가 반드시 생길 것입니다.  또한 태릉입구역 주변과 동부간선도로 주변의 교통체증은 1년 365일 있는 일로 누구나 알고 있는 일인지라 여기서 더 이상 거론은 하지 않겠습니다.  태릉골프장 절대로 택지로 개발하면 안 됩니다. 이런 민원을 올리면 이 동네 집값은 지금도 최하인데 집값 떨어지면 안 되니 민원 올리지 말라고 합니다. 그러나 이 문제는 묵동, 공릉동 주민의 생존의 문제입니다. 녹지 유지하게 해 주시고 개발을 막아주세요. 간곡히 부탁드립니다. 
                                
2 페이지 내용 수집 시작합니다 ==================

19 : 
Q. 상담내용
                                    현재 용산구에 거주중입니다. 부모님이 서초구 방배동에 거주하고 계셔서 매주 방문한는데. 매 격주 마다 서초구청에서 부모님들은 위한 마스크와 손세정제를 집에 놓고 가십니다. 서울시에서 제공한 마스크와 서초구청에서 제공하는마스크 별도로 부모님 각각 2묶음을 주고 갑니다. 현재 용산구 보광동 주변에는 장모님고 처가집 삼촌들 모두 65세이상이신데, 단 한번도 서울시나 용산구청에서 주는 마스크를 받아 본적이 없습니다.  용산구에서는 제공을 못한다고 하여도 서울시에서 제공하는 마스크는 모든 구에 균등하게 제공되어야하는 것이 맞습니다. 왜 서초구에는 제공이 되고 용산구는 없는지 정확한 이유를 알고 싶습니다. 
                                
20 : 
Q. 상담내용
                                    안녕하십니까? 해마다 우기가 되면 호우경보가 울릴까 봐, 묵동천이 넘칠까 조마조마 마음 졸이고 잠을 설치며 살고 있는 중랑구 묵동 주민입니다.  태릉골프장 아래에 있는 묵동천은 집중호우 때 물의 양이 급속도로 증가되어 수위가 순식간에 상승되는 지역입니다. 빗물이 묵동천 제방 위협할 때가 여러 번 있었으며 묵동천 제방 아래 지역은 하수 역류 등으로 집중호우 때  침수 피해가 조금 있었던 적도 있습니다.  골프장의 잔디와 흙은 빗물을 흡수하여 땅속에 저장하는 중요한 역할을 하고 있습니다. 태릉골프장 개발이 되어 일만 채 이상의 아파트 건물이 들어서면 아스팔트 포장으로 빗물이 땅속으로 스며들지 못해 많은 양의 빗물이 순식간에 흘러 들어와 묵동천이 감당하지 못해 넘칠 것입니다.  또한 일만 채의 아파트에서 동시에 쏟아져 나오는 생활하수의 양도 엄청나서 집중 호우 시  제방을 위협할 것이라 생각됩니다.  지금 한창 개발되고 있는 신내역 주변 아파트도 내년부터 입주를 합니다. 이곳 역시 많은 생활하수와 빗물의 양이 증가하여 묵동천을 위협할 것입

33 : 
Q. 상담내용
                                    금번  35층 이상 주택건설에 반대한 서울시임직원 여러분께 감사드리오며태릉 등  서울에 얼마남아있지 않은  그린밸트를 그대로 보존하여 서울 우리 후세들에게 물려 주세요다시는 더이상 서울이 천박한 도시가 되지 않게 힘써주세요
                                
34 : 
Q. 상담내용
                                    안녕하세요?752번이 서리풀터널로 연장되는 방안을 서울시에서 추진하고 있음은 잘 알고 있습니다.그러나 서리풀터널을 통과한 후 서초역부터 강남역까지 구간은 극심 정체되는 교통체증이 심한 구간입니다.752번이 이 구간으로 노선연장됨에 따라, 배차간격 증가 및 불규칙 현상이 심해질 것으로 파악됩니다.752번은 대당승객이 900명 정도 되는 노선으로 대당승객이 많은 편인 노선입니다.또한 숭실대와 사당로에서 총신대입구역에 가는 단독노선이고, 사당로에서 노량진역에 가는 단독노선이여서 승객이 많아 배차간격이 길어질 경우 이용승객들의 큰 불편이 발생된다고 생각합니다.뿐만 아니라 구산동에서 서대문, 경찰청, 서울역까지 가장 빠른 시간 안에 접근하는 노선이기 때문에 출근시간 구산동 종점부터 승차하는 승객들이 굉장히 많습니다.그래서 출근시간 홍제역에 가면 미탑승이 발생하기 시작해 서대문까지 미탑승이 발생하여 그 승객들이 서대문, 경찰청, 서울역에서 다 하차합니다. 특히 752번이 거치는 녹번역에 대규모아파트단지가 입주할 예정이기에 출근시간 혼잡 가중과 미탑승 증가 현상이 우려됩니다.752번의 운수사에는 예비차가 총 14대이고, 그 중 잉여예비차는 2대입니다.752번이 서리풀터널로 노선이 연장되면서 잉여예비차 2대를 752번에 증차함으로써 752번의 배차간격 증가를 최소화하는 방안 등을 검토해주시기 바랍니다.이겨라 주무관님께서 답변해주세요.
                                
35 : 
Q. 상담내용
  

47 : 
Q. 상담내용
                                    그동안 페미니스트, 82년생 공지영 영화를 보며 여성인권을 위해 힘쓰겠다고 하던 박원순 시장이 자살로 안타까운 선택을 했습니다. 고인의 죽음은 안타깝습니다. 그러나 현재 성추행 사건의 고소가 있었고 피해자가 있는 상태입니다. 5일의 장례를 서울시 세금으로 진행하는 것은 문제가 있습니다. 피해자의 상황은 전혀 고려하지 않은채 오히려 가해자를 피해자처럼 만들고 있습니다. 실제로 박원순 시장의 지지자들이 피해자를 위협하고 조롱하는 악성댓글을 다는 일들이 일어나고 있습니다. 가정의 장례로 치뤄지면 될 것을 서울시의 세금으로 하는 것도 문제지만, 힘없는 개인, 그것도 여성을 완전히 파괴시키는 일이라 생각됩니다. 그동안 박원순 시장의 뜻대로 여성인권에 앞섰던만큼 피해자를 위해서라도 서울시 특별장을 하지 말아주세요. 
                                
48 : 
Q. 상담내용
                                    코로나 19로 사회적 거리두기, 사적 만남의 제한, 재택근무 등 집에 있는 시간이 많아졌습니다. 집에 있는 시간을 잘 활용하기 위한 방법으로 독서를 빼놓을 수 없습니다. 독서를 위해서 지역 도서관에서 책을 대출받고 싶은데, 지역 도서관들은 코로나 19를 핑계로 문을 닫거나 아주 일부만 개방을 하고 있는 상황입니다. 지역도서관들은 크게 세 가지 활동을 합니다. 먼저 독서 혹은 지역주민들을 위한 프로그램입니다. 그리고 열람실 운영 마지막으로 책의 대출(대차대조 등 포함)입니다. 현재 코로나 상황에서 프로그램과 열람실 운영은 할 수 없고, 책 대출업무만 가능한 상황은 이해가 됩니다. 따라서 프로그램을 담당하거나 열람실을 운영하는 직원들의 업무가 대폭 줄어들었음에도 불구하고 책 대출업무마저 중단하거나 업무시간을 대폭 줄여서 운영하는 지역도서관들이 태반입니다. 직원들 대부분이 코로나 19를 핑계로 놀고있거나 재택근무 혹은 휴가로 시간